In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_ml_control
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
%load_latex_macros

In [ ]:
%autoreload
import os
import warnings
from dataclasses import dataclass
from typing import Protocol

import casadi
import do_mpc
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from gymnasium import Env
from IPython.display import HTML
from ipywidgets import interact, widgets
from matplotlib.animation import FuncAnimation
from numpy.typing import NDArray

from training_ml_control.environment import (
    create_inverted_pendulum_environment,
)

from training_ml_control.shortest_path_problem import (
    create_shortest_path_graph,
    plot_shortest_path_graph,
    plot_all_paths_graph,
)

from training_ml_control.plots import (
    create_inverted_pendulum_environment,
    plot_inverted_pendulum_results,
    animate_mass_spring_damper_simulation,
    animate_inverted_pendulum_simulation,
    animate_full_inverted_pendulum_simulation,
)

from training_ml_control.nb_utils import (
    display_array,
    show_video,
)

warnings.simplefilter("ignore", UserWarning)
sns.set_theme()
plt.rcParams["figure.figsize"] = [9, 5]
# This is needed because inside docker the rendering of mujoco environments may not work.
render_mode = "rgb_array" if os.environ.get("DISPLAY") else None

### Linearized Model

In [ ]:
env = create_inverted_pendulum_environment()

ip_parameters = InvertedPendulumParameters(
    l=env.model.geom_pos[2, 2],
    m=env.model.body_mass[2],
    M=env.model.body_mass[1],
)
# Dynamics matrix
A = np.array(
    [
        [0, 1],
        [
            (ip_parameters.M + ip_parameters.m)
            * ip_parameters.g
            / (ip_parameters.m * ip_parameters.l),
            0,
        ],
    ]
)
# Input matrix
B = np.array(
    [[0, -ip_parameters.gamma / (ip_parameters.M * ip_parameters.l)]]
).transpose()
# Output matrices
C = np.array([[1, 0], [0, 1]])
D = np.zeros(2)

inverted_pendulum_lin = do_mpc.model.LinearModel("continuous")
theta = inverted_pendulum_lin.set_variable(var_type="_x", var_name="theta")
dtheta = inverted_pendulum_lin.set_variable(var_type="_x", var_name="dtheta")
inverted_pendulum_lin.set_variable(var_type="_u", var_name="force")

# Inertia
J = (ip_parameters.m * ip_parameters.l**2) / 3
# Energies
E_kin = 0.5 * J * dtheta**2 + 0.5 * ip_parameters.m * (
    (ip_parameters.l * dtheta * casadi.cos(theta)) ** 2
    + (ip_parameters.l * dtheta * casadi.sin(theta)) ** 2
)
E_pot = ip_parameters.m * ip_parameters.g * ip_parameters.l * casadi.cos(theta)
inverted_pendulum_lin.set_expression("E_kinetic", E_kin)
inverted_pendulum_lin.set_expression("E_potential", E_pot)

inverted_pendulum_lin.setup(A, B, C, D)
env = create_inverted_pendulum_environment()
inverted_pendulum_lin = inverted_pendulum_lin.discretize(env.dt)

### Non-Linear Model

In [ ]:
inverted_pendulum = do_mpc.model.Model("continuous")

pos = inverted_pendulum.set_variable(var_type="_x", var_name="position")
theta = inverted_pendulum.set_variable(var_type="_x", var_name="theta")
dpos = inverted_pendulum.set_variable(var_type="_x", var_name="velocity")
dtheta = inverted_pendulum.set_variable(var_type="_x", var_name="dtheta")
u = inverted_pendulum.set_variable(var_type="_u", var_name="force")

k = 1 / 3
numerator = (
    (ip_parameters.M + ip_parameters.m) * ip_parameters.g * casadi.sin(theta)
    - casadi.cos(theta)
    * (
        ip_parameters.gamma * u
        + ip_parameters.m * ip_parameters.l * dtheta**2 * casadi.sin(theta)
        - ip_parameters.mu_c * dpos
    )
    - (ip_parameters.M + ip_parameters.m)
    / (ip_parameters.m * ip_parameters.l)
    * ip_parameters.mu_p
    * dtheta
)
denominator = (1 + k) * (
    ip_parameters.M + ip_parameters.m
) * ip_parameters.l - ip_parameters.m * ip_parameters.l * casadi.cos(theta) ** 2
ddtheta = numerator / denominator

numerator = (
    ip_parameters.m * ip_parameters.g * casadi.cos(theta) * casadi.sin(theta)
    - (1 + k)
    * (
        ip_parameters.gamma * u
        + ip_parameters.m * ip_parameters.l * dtheta**2 * casadi.sin(theta)
        - ip_parameters.mu_c * dpos
    )
    - ip_parameters.mu_p * dtheta * casadi.cos(theta) / ip_parameters.l
)
denominator = ip_parameters.m * casadi.cos(theta) ** 2 - (1 + k) * (
    ip_parameters.M + ip_parameters.m
)
ddpos = numerator / denominator

inverted_pendulum.set_rhs("position", dpos)
inverted_pendulum.set_rhs("theta", dtheta)
inverted_pendulum.set_rhs("velocity", ddpos)
inverted_pendulum.set_rhs("dtheta", ddtheta)

# Inertia
J = (ip_parameters.m * ip_parameters.l**2) / 3
# Energies
E_kin = (
    0.5 * J * dtheta**2
    + 0.5 * ip_parameters.M * dpos**2
    + 0.5
    * ip_parameters.m
    * (
        (dpos + ip_parameters.l * dtheta * casadi.cos(theta)) ** 2
        + (ip_parameters.l * dtheta * casadi.sin(theta)) ** 2
    )
)
E_pot = ip_parameters.m * ip_parameters.g * ip_parameters.l * casadi.cos(theta)
inverted_pendulum.set_expression("E_kinetic", E_kin)
inverted_pendulum.set_expression("E_potential", E_pot)

inverted_pendulum.setup()

### Simulators

In [ ]:
env = create_inverted_pendulum_environment()
linear_simulator = do_mpc.simulator.Simulator(inverted_pendulum_lin)
linear_simulator.set_param(t_step=env.dt)
linear_simulator.setup()

env = create_inverted_pendulum_environment(cutoff_angle=np.inf)
nonlinear_simulator = do_mpc.simulator.Simulator(inverted_pendulum)
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    "integration_tool": "idas",
    "abstol": 1e-8,
    "reltol": 1e-8,
    "t_step": env.dt,
}
nonlinear_simulator.set_param(**params_simulator)

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::

## Linear Quadratic Regulator

While solving the optimal control problem (OCP) is very hard in general, there are a few very important special cases where the solutions are very accessible. Most of these involve variants on the case of linear dynamics and quadratic cost. The simplest case, called the linear quadratic regulator (LQR), is formulated as stabilizing a time-invariant linear system to the origin can be solved analytically.

For a discrete-time linear system described by:

$$
\mathbf{x}_{t+1} = A \mathbf{x}_t + B \mathbf{u}_t
$$

where $x\in \mathbb {R} ^{n}$ (that is, $x$ is an $n$-dimensional real-valued vector) is the state of the system and $u\in \mathbb {R} ^{m}$ is the control input. Given a quadratic cost function for the system in the infinite-horizon case, defined as:

$$
J(\mathbf{x}_0, \mathbf{u}) = \sum \limits _{k = 0}^{\infty} \mathbf{x}_k^{T}Q \mathbf{x}_k + \mathbf{u}_k^{T} R \mathbf{u}_k
$$

With $Q = Q^T \succeq 0$, $R = R^T \succeq 0$.

In both cases, the control law that minizes the cost is given by:

$$
u_k = -K x_k
$$

With: $K = (R + B^T P B)^{-1} B^T P B$

and $P$ is found by solving the discrete time algebraic Riccati equation (DARE):

$$
Q + A^{T}PA-(A^{T}PB)(R+B^{T}PB)^{-1}(B^{T}PA) = P.
$$

:::{figure} _static/images/30_lqr_block_diagram.svg
:width: 60%
LQR Block Diagram.
:::

In [ ]:
def create_lqs_controller(model, t_step: float, n_horizon: int | None = None):
    lqr = do_mpc.controller.LQR(mass_spring_damper)
    lqr.settings.t_step = env.dt
    lqr.settings.n_horizon = None
    return lqr

### Mass-Spring-Damper

Now, we design Linear Quadratic Regulator for the Mass-Spring-Damper model. First, we create an instance of the LQR class.

In [ ]:
lqr = do_mpc.controller.LQR(mass_spring_damper)

In [ ]:
env = create_mass_spring_damper_environment(render_mode=render_mode)
lqr.settings.t_step = env.dt
lqr.settings.n_horizon = None  # infinite horizon

In [ ]:
Q = np.diag([1000, 0])
R = np.diag([0])
display_array("Q", Q)
display_array("R", R)
lqr.set_objective(Q=Q, R=R)

In [ ]:
lqr.setup()

In [ ]:
xss = np.array([0.1, 0.0]).reshape(-1, 1)
lqr.set_setpoint(xss)

#### SImulation

Now, we simulate the closed-loop system for 50 steps:

In [ ]:
lqr.reset_history()
mass_spring_damper_simulator.reset_history()
x0 = np.zeros((2, 1))
mass_spring_damper_simulator.x0 = x0
for _ in range(50):
    u0 = lqr.make_step(x0)
    x0 = mass_spring_damper_simulator.make_step(u0)

In [ ]:
animate_mass_spring_damper_simulation(lqr.data)

#### Evaluation

Finally, we evaluate the controller on the actual environment.

In [ ]:
class LQRController:
    def __init__(self, lqr: do_mpc.controller.LQR) -> None:
        self.lqr = lqr
        self.lqr.reset_history()

    def act(self, observation: NDArray) -> NDArray:
        return lqr.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
max_steps = 100
env = create_mass_spring_damper_environment(
    render_mode=render_mode, max_steps=max_steps
)
controller = LQRController(lqr)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
plt.close()
fig, ax, graphics = do_mpc.graphics.default_plot(lqr.data)
ax[0].hlines(xss[0], lqr.data._time[0], lqr.data._time[-1], "r");

::::{exercise} LQR Controller
:label: lqr-controller
Design an LQR controller to keep the inverted pendulum by following these steps:

1. Create an instance of `do_mpc.controller.LQR` by using the `create_lqs_controller` function and passing the appropriate `t_step` and `n_horizon` values.
2. Define the $Q$ and $R$ matrices and set the controller's objective using its `set_objective` method.
3. Call the setup
::::

:::{solution} lqr-controller
:::

In [ ]:
# Write your solution here

::::{solution} lqr-controller
:class: dropdown

:::{code-cell}
lqr = do_mpc.controller.LQR(inverted_pendulum_lin)
:::

We choose an infinite prediction horizon (`n_horizon = None`), the time step `t_step = 0.04s` second (which is the same as the environment's timestep).

:::{code-cell} python3
env = create_inverted_pendulum_environment()
lqr.settings.t_step = env.dt
lqr.settings.n_horizon = None  # infinite horizon
:::

The goal is to keep the inverted pendulum uprightThe goal is to keep the inverted pendulum upright. For that we define the following objective.. For that we define the following objective.

:::{code-cell} python3
Q = np.diag([100, 1])
R = np.diag([10])
display_array("Q", Q)
display_array("R", R)
lqr.set_objective(Q=Q, R=R)
:::

We then complete the LQR setup.

:::{code-cell} python3
lqr.setup()
:::

Finally, we set the desired setpoint.

:::{code-cell} python3
xss = np.array([0.0, 0.0]).reshape(-1, 1)
lqr.set_setpoint(xss)
:::

#### Simulation

:::{code-cell}
lqr.reset_history()
inverted_pendulum_lin_simulator.reset_history()
x0 = np.zeros((2, 1))
x0[0] = 0.01
inverted_pendulum_lin_simulator.x0 = x0

for k in range(50):
    u0 = lqr.make_step(x0)
    x0 = inverted_pendulum_lin_simulator.make_step(u0)

animate_inverted_pendulum_simulation(lqr.data)
:::

#### Evaluation

:::{code-cell}
class LQRController:
    def __init__(self, lqr: do_mpc.controller.LQR) -> None:
        self.lqr = lqr
        self.lqr.reset_history()

    def act(self, observation: NDArray) -> NDArray:
        return lqr.make_step(observation[[1, 3]].reshape(-1, 1)).ravel()
:::

:::{code-cell}
max_steps = 500
env = create_inverted_pendulum_environment(render_mode=render_mode, max_steps=max_steps)
controller = LQRController(lqr)
results = simulate_environment(env, max_steps=max_steps, controller=controller)
show_video(results.frames, fps=1 / env.dt)
:::

:::{code-cell}
plt.close()
fig, ax, graphics = do_mpc.graphics.default_plot(lqr.data, aux_list=[])
ax[0].hlines(xss[0], lqr.data._time[0], lqr.data._time[-1], "r");
:::

::::

In [ ]:
lqr.reset_history()
inverted_pendulum_lin_simulator.reset_history()
x0 = np.zeros((2, 1))
x0[0] = 0.01
inverted_pendulum_lin_simulator.x0 = x0
for k in range(50):
    u0 = lqr.make_step(x0)
    x0 = inverted_pendulum_lin_simulator.make_step(u0)

In [ ]:
animate_inverted_pendulum_simulation(lqr.data)

In [ ]:
class LQRController:
    def __init__(self, lqr: do_mpc.controller.LQR) -> None:
        self.lqr = lqr
        self.lqr.reset_history()

    def act(self, observation: NDArray) -> NDArray:
        return lqr.make_step(observation[[1, 3]].reshape(-1, 1)).ravel()

In [ ]:
max_steps = 500
env = create_inverted_pendulum_environment(render_mode=render_mode, max_steps=max_steps)
controller = LQRController(lqr)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
plt.close()
fig, ax, graphics = do_mpc.graphics.default_plot(lqr.data, aux_list=[])
ax[0].hlines(xss[0], lqr.data._time[0], lqr.data._time[-1], "r");

:::{solution-end}
:::

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::